# 200601_ベストスコア1.4672364_RF
 train,testデータに分割して過学習しないようパラメータチューニングして、その後2017年のデータすべて使用してモデル作成。
 
 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=30, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=50,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=88, verbose=0,
                       warm_start=False)

# ライブラリの読み込み

In [15]:
#データ加工でよく使うやつ
import datetime  # 日付,時刻の処理
import yaml  # yamlファイル操作
import re  # 正規表現操作
import shutil  # ファイル操作
import glob  # ファイル操作
import os  # ファイル操作
import seaborn as sns  # グラフ描画
import pandas as pd  # データ加工入出力,DataFlame
import numpy as np  # 行列計算
import scipy as sp  # 科学技術計算
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

#モデリング
from sklearn.model_selection import train_test_split # 訓練データとテストデータの分割を行うライブラリ
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc  # 評価指標
# KFold交差検証のライブラリ、交差検証のスコアを求めるライブラリ、および交差検証による予測に関するライブラリをimport
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb

In [16]:
df = pd.read_pickle('../yakiu_rawdata/df_merge.pkl').drop(['データ内連番','N'],axis=1)

In [17]:
#df.イニング = df.イニング.astype(object)

# 分割データの作成
- カテゴリ変数はダミー化
- 目的変数は数値でよくて、分類のモデルを選べばOK
- 欠損があると回らない。

In [18]:
#category有のとき(事前にint型に変更必要)
df2 = pd.get_dummies(df,drop_first=True)
df2.head(2)

,球種,年度,試合内連番,試合内投球数,イニング,イニング内打席数,打席内投球数,投手登板順,投手試合内対戦打者数,投手試合内投球数,投手イニング内投球数,打者打順,打者試合内打席数,pitch_team_score,month,P_身長,P_体重,P_年俸,P_age,P_age_split,P_BMI,B_身長,B_体重,B_年俸,B_age,B_age_split,B_BMI,表裏_裏,投手投球左右_左,投手役割_救援,打者打席左右_左,打者守備位置_三,打者守備位置_中,打者守備位置_二,打者守備位置_右,打者守備位置_左,打者守備位置_打,打者守備位置_投,打者守備位置_指,打者守備位置_捕,打者守備位置_無,打者守備位置_遊,プレイ前走者状況_12_,プレイ前走者状況_1_3,プレイ前走者状況_1__,プレイ前走者状況__23,プレイ前走者状況__2_,プレイ前走者状況___3,プレイ前走者状況____,count_0-0-1,count_0-0-2,count_0-1-0,count_0-1-1,count_0-1-2,count_0-2-0,count_0-2-1,count_0-2-2,count_0-3-0,count_0-3-1,count_0-3-2,count_1-0-0,count_1-0-1,count_1-0-2,count_1-1-0,count_1-1-1,count_1-1-2,count_1-2-0,count_1-2-1,count_1-2-2,count_1-3-0,count_1-3-1,count_1-3-2,count_2-0-0,count_2-0-1,count_2-0-2,count_2-1-0,count_2-1-1,count_2-1-2,count_2-2-0,count_2-2-1,count_2-2-2,count_2-3-0,count_2-3-1,count_2-3-2,P_血液型_AB,P_血液型_B,P_血液型_O,P_血液型_不明,P_出身地域_北米,P_出身地域_日本,B_血液型_AB,B_血液型_B,B_血液型_O,B_血液型_不明,B_出身地域_北米,B_出身地域_日本
0,0.0,2017,1,1,1,1,1,1,1,1,1,1,1,0,3,196,100,24000,29,5,26.03082,176,74,15000,32,6,23.889463,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1
1,0.0,2017,2,2,1,1,2,1,1,2,2,1,1,0,3,196,100,24000,29,5,26.03082,176,74,15000,32,6,23.889463,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1


In [19]:
df_test_merge = df2[df2['球種'].isnull()].drop(['球種'],axis=1).reset_index(drop = True)
df_train_merge = df2.dropna(subset=['球種'])

In [20]:
df_train_merge = df_train_merge.astype('int64')
df_train_merge = pd.DataFrame(df_train_merge)
df_test_merge = df_test_merge.astype('int64')
df_test_merge = pd.DataFrame(df_test_merge)

In [21]:
df_train_merge_x = pd.DataFrame(df_train_merge.drop('球種',axis=1))
df_train_merge_y = pd.DataFrame(df_train_merge.loc[:,'球種'])

In [22]:
len(df_train_merge)

257117

In [23]:
len(df_test_merge)

521650

In [24]:
#label比率を合わせて分割
df_train,df_test=train_test_split(df_train_merge, test_size=0.3, random_state = 88,  stratify=df_train_merge['球種']) 

In [25]:
df_train_y = pd.DataFrame(df_train.loc[:,'球種'])
df_train_x = pd.DataFrame(df_train.drop('球種',axis=1))
df_test_y = pd.DataFrame(df_test.loc[:,'球種'])
df_test_x = pd.DataFrame(df_test.drop('球種',axis=1))

In [26]:
df_train_merge_x.head(1)

,年度,試合内連番,試合内投球数,イニング,イニング内打席数,打席内投球数,投手登板順,投手試合内対戦打者数,投手試合内投球数,投手イニング内投球数,打者打順,打者試合内打席数,pitch_team_score,month,P_身長,P_体重,P_年俸,P_age,P_age_split,P_BMI,B_身長,B_体重,B_年俸,B_age,B_age_split,B_BMI,表裏_裏,投手投球左右_左,投手役割_救援,打者打席左右_左,打者守備位置_三,打者守備位置_中,打者守備位置_二,打者守備位置_右,打者守備位置_左,打者守備位置_打,打者守備位置_投,打者守備位置_指,打者守備位置_捕,打者守備位置_無,打者守備位置_遊,プレイ前走者状況_12_,プレイ前走者状況_1_3,プレイ前走者状況_1__,プレイ前走者状況__23,プレイ前走者状況__2_,プレイ前走者状況___3,プレイ前走者状況____,count_0-0-1,count_0-0-2,count_0-1-0,count_0-1-1,count_0-1-2,count_0-2-0,count_0-2-1,count_0-2-2,count_0-3-0,count_0-3-1,count_0-3-2,count_1-0-0,count_1-0-1,count_1-0-2,count_1-1-0,count_1-1-1,count_1-1-2,count_1-2-0,count_1-2-1,count_1-2-2,count_1-3-0,count_1-3-1,count_1-3-2,count_2-0-0,count_2-0-1,count_2-0-2,count_2-1-0,count_2-1-1,count_2-1-2,count_2-2-0,count_2-2-1,count_2-2-2,count_2-3-0,count_2-3-1,count_2-3-2,P_血液型_AB,P_血液型_B,P_血液型_O,P_血液型_不明,P_出身地域_北米,P_出身地域_日本,B_血液型_AB,B_血液型_B,B_血液型_O,B_血液型_不明,B_出身地域_北米,B_出身地域_日本
0,2017,1,1,1,1,1,1,1,1,1,1,1,0,3,196,100,24000,29,5,26,176,74,15000,32,6,23,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1


In [27]:
from sklearn.ensemble import RandomForestClassifier as RFC 
from sklearn.model_selection import train_test_split, GridSearchCV

### 過学習しまくりなのでグリッドサーチしてパラメータを設定する。
- https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [28]:
search_params = {
     'n_estimators'      : [ 100],
#      'max_features'      : [3, 5, 10, 15, 20],
      'random_state'      : [88],
      'n_jobs'            : [-1],
      'min_samples_split' : [ 50],
      'max_depth'         : [ 30]
}

In [29]:
gs = GridSearchCV(RFC(),           # 対象の機械学習モデル
                  search_params,   # 探索パラメタ辞書
                  cv=3,            # クロスバリデーションの分割数
                  verbose=True,    # ログ表示
                  n_jobs=-1)       # 並列処理
gs.fit(df_train_merge_x, df_train_merge_y)
 
print(gs.best_estimator_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   46.6s finished
C:\Users\e11541\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=30, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=50,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=88, verbose=0,
                       warm_start=False)


In [30]:
# 最高性能のモデルを取得し、テストデータを分類
best = gs.best_estimator_
#predict_train_label =  pd.DataFrame(best.predict_proba(df_train_x))
#predict_test_label =  pd.DataFrame(best.predict_proba(df_test_x))

In [31]:
best

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=30, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=50,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=88, verbose=0,
                       warm_start=False)

In [32]:
#特徴量の重要度
feature = pd.DataFrame(best.feature_importances_,columns=['importance'])
label = pd.DataFrame(df_train_merge_x.columns[0:],columns=['label'])
importance = pd.concat([label,feature],axis=1)
importance.sort_values('importance',ascending = False)

,label,importance
16,P_年俸,0.072464
14,P_身長,0.069685
15,P_体重,0.060120
17,P_age,0.056808
19,P_BMI,0.052347
2,試合内投球数,0.036512
1,試合内連番,0.036226
8,投手試合内投球数,0.033924
9,投手イニング内投球数,0.027465
22,B_年俸,0.027005


### グリッドサーチで過学習が抑えられたので、Submitデータを作成する。

In [34]:
submit_label = best.predict_proba(df_test_merge)

In [35]:
submit_label = pd.DataFrame(submit_label)
submit_label

,0,1,2,3,4,5,6,7
0,0.427983,0.056965,0.121226,0.228562,0.024880,0.006843,0.020214,0.113328
1,0.352623,0.082624,0.107434,0.284389,0.025485,0.005984,0.019965,0.121496
2,0.349261,0.105895,0.152878,0.218658,0.044896,0.012532,0.016482,0.099398
3,0.359276,0.081470,0.188688,0.223056,0.041803,0.012897,0.014651,0.078159
4,0.385731,0.066091,0.164603,0.192304,0.070769,0.022448,0.013092,0.084961
...,...,...,...,...,...,...,...,...
521645,0.472424,0.070844,0.112507,0.070142,0.234865,0.018158,0.005321,0.015739
521646,0.455484,0.094016,0.122151,0.085985,0.199497,0.027378,0.000286,0.015203
521647,0.481163,0.059611,0.104104,0.083352,0.232697,0.026094,0.000278,0.012701
521648,0.513983,0.052354,0.100684,0.085814,0.211861,0.022512,0.002150,0.010642


In [63]:
submit_label.to_csv("200601_submit_RFC_GS3.csv",header=False,index=True)